In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
# %aimport abcutils

In [ ]:
import pandas
import abcutils

In [ ]:
df = abcutils.load_and_synthesize_csv('tmp2.csv.gz')

In [ ]:
df.head()

In [ ]:
filters = [
    df['darshan_app'] == 'ior', # only want IOR jobs
    df['darshan_biggest_api_bytes'] > 2.0**40, # only want jobs that did more than 1 TiB of I/O
]

abcutils.apply_filters(df, filters, verbose=True)

In [ ]:
target_col = 'darshan_agg_perf_by_slowest_posix'
group_by_cols = ['darshan_app', '_file_system', 'darshan_fpp_or_ssf_job', 'darshan_read_or_write_job']
new_col_base = 'darshan_normalized_perf'

# modifies the dataframe in-place; returns nothing
abcutils.normalize_column(
    dataframe=df,
    target_col=target_col,
    group_by_cols=group_by_cols,
    new_col_base=new_col_base)

In [ ]:
df[group_by_cols + [target_col]].head()

In [ ]:
df[group_by_cols + [target_col] + [new_col_base + "_by_max"]].head()

## Correlation Matrix

In [ ]:
ax, correlations = abcutils.plot.correlation_matrix(df, fontsize=14)
ax.figure.set_size_inches(20, 20)

## Correlation Table

In [ ]:
# pandas.options.display.max_rows = 9999

correlation = abcutils.correlation.calc_correlation_vector(df, correlate_with='darshan_normalized_perf_by_max')

filtered_correlations = abcutils.apply_filters(correlation, [correlation['p-value'] < 1.0e-5], verbose=True)
filtered_correlations

In [ ]:
ax = abcutils.plot.correlation_vector_table(filtered_correlations)
ax.figure.set_size_inches(4, 0.4 * len(filtered_correlations))